In [1]:
# !wget https://github.com/mesolitica/llama2-embedding/raw/main/test-set/b.cari.com.my-dataset-bge-test.sample.json
# !wget https://huggingface.co/datasets/mesolitica/embedding-pair-mining/resolve/main/b.cari.com.my-dataset-bge-test.jsonl

In [2]:
!nvidia-smi

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Mon Jan 22 05:14:50 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   29C    P0    68W / 300W |  54690MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000002:00:00.0 Off |                    0 |
| N/A   

In [6]:
import json
import openai
import torch
from tqdm import tqdm

In [7]:
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

model = AutoModel.from_pretrained(
    'mesolitica/mistral-embedding-191m-8k-contrastive', 
    use_flash_attention_2 = True,
    trust_remote_code = True,
    torch_dtype = torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/mistral-embedding-191m-8k-contrastive')

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [8]:
_ = model.cuda()

In [9]:
with open('b.cari.com.my-dataset-bge-test.sample.json') as fopen:
    rev_data = json.load(fopen)

In [10]:
vectors = {}
for k, v in tqdm(rev_data.items()):
    try:
        padded = tokenizer([k],return_tensors = 'pt', padding = True)
        for k_ in padded:
            padded[k_] = padded[k_].cuda()

        vectors[k] = model.encode(padded).cpu().detach().numpy()[0]
    except:
        pass

100%|██████████| 18755/18755 [05:42<00:00, 54.76it/s]


In [11]:
no_string = {no: k for no, (k, v) in enumerate(vectors.items())}
string_no = {v: k for k, v in no_string.items()}
no_string[0]

'Aku yg x reti masak ni nak suruh buat sendiri? Uwaaaa kt mana leh dpt ni..kedai siam xdop kaa'

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [16]:
np_vectors = np.array(list(vectors.values()))
np_vectors.shape

(18755, 768)

In [17]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}
total = 0
with open('b.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        query_no = string_no.get(query)
        if query_no is None:
            continue
        for s in l['pos']:
            s = s.strip()
            v_s = vectors.get(s)
            s_no = string_no.get(s)
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            argsort = np.argsort(cosine_similarity(v_s, np_vectors)[0])[::-1]
            for k in tops.keys():
                if s_no in argsort[:k]:
                    k_ = k + 1
                else:
                    k_ = k
                if query_no in argsort[:k_]:
                    tops[k] += 1
            total += 1
            
for k, v in tops.items():
    print(k, v / total)

9541it [01:55, 82.89it/s]  

1 0.2564182842830307
3 0.46900438321853477
5 0.5309956167814652
10 0.6036318096430808
